In [ ]:
from __future__ import annotations
import math, random
from typing import List, Tuple
from PIL import Image, ImageDraw

# A point is simply an (x, y) coordinate
Point = Tuple[float, float]
# A polygon is a list of such points
Polygon = List[Point]

class PookalamExact:
    

    def __init__(self, size: int = 800):
        # Canvas size and center
        self.size = size
        self.cx = self.cy = size / 2

        # Colors
        self.COL_YELLOW   = "#FFD93B"
        self.COL_ORANGE   = "#FD6A00"
        self.COL_RED      = "#E63946"
        self.COL_DARKRED  = "#8B0000"
        self.COL_GREEN    = "#00563B"
        self.COL_BROWN    = "#7B3F00"
        self.COL_CREAM    = "#F5F5DC"
        self.COL_VIOLET   = "#7F00FF"
        self.COL_LGREEN = "#4CAF50"
        self.COL_Blue     = "#00308F"
        self.Col_Pink     = "#FF69B4"

        # Background 
        self.BG       = "#FFFFFF"
        self.OUTLINE  = "#000000"

   
    #  Geometry 
    def polar(self, r: float, angle_deg: float) -> Point:
        """Convert polar coordinates (r, angle) into Cartesian (x, y)."""
        a = math.radians(angle_deg)
        return (self.cx + r * math.cos(a), self.cy + r * math.sin(a))

    def circle_bbox(self, r: float):
        """Return bounding box for a circle of radius r centered at canvas center."""
        return (int(self.cx - r), int(self.cy - r),
                int(self.cx + r), int(self.cy + r))

    
    #  Shape Generators
    
    def petals(self):
        """Generate 8 orange petals radiating from the center."""
        petals: List[Polygon] = []
        r_tip, r_mid = 150, 70   # outer tip radius, inner mid radius

        for i in range(8):
            ang = i * 45.0   # evenly spaced at 45 degrees
            center = (self.cx, self.cy)
            p1 = self.polar(r_tip, ang + 22.5)
            p2 = self.polar(r_mid, ang)
            p3 = self.polar(r_tip, ang - 22.5)
            petals.append([center, p1, p2, p3])
        return petals

    def tri_ring(self):
        
        shapes = []
        inner_r, outer_r = 200, 260

        for i in range(16):
            ang = i * 22.5
            apex = self.polar(inner_r, ang)
            b1   = self.polar(outer_r, ang - 11.25)
            b2   = self.polar(outer_r, ang + 11.25)

            col = self.Col_Pink if i % 2 == 0 else self.COL_VIOLET
            shapes.append(([apex, b1, b2], col))
        return shapes

    def outer_tri_ring(self):
        
        shapes = []
        inner_r, outer_r = 400, 440   # base radius at 400, tips at 440
        count = 20                   # number of triangles
        step = 360.0 / count

        for i in range(count):
            ang = i * step
            base1 = self.polar(inner_r, ang - step/2)
            base2 = self.polar(inner_r, ang + step/2)
            tip   = self.polar(outer_r, ang)

            col = self.COL_RED if i % 2 == 0 else self.Col_Pink
            shapes.append(([base1, base2, tip], col))
        return shapes

    def checker_ring(self, count, r_inner, r_outer, colors):
        """Generate checkerboard-style ring segments."""
        result = []
        step = 360.0 / count

        for i in range(count):
            ang = i * step
            p1 = self.polar(r_inner, ang)
            p2 = self.polar(r_outer, ang)
            p3 = self.polar(r_outer, ang + step)
            p4 = self.polar(r_inner, ang + step)

            col = colors[i % len(colors)]
            result.append(([p1, p2, p3, p4], col))
        return result

    
    #  Effects
    def add_grain(self, img: Image.Image, intensity: int = 30) -> Image.Image:
        noise = Image.new("RGB", img.size)
        pixels = noise.load()

        for y in range(img.size[1]):
            for x in range(img.size[0]):
                val = random.randint(-intensity, intensity)
                r, g, b = img.getpixel((x, y))

                r = max(0, min(255, r + val))
                g = max(0, min(255, g + val))
                b = max(0, min(255, b + val))

                pixels[x, y] = (r, g, b)

        return noise

    
    #  Main Draw Function
    def draw(self, out_path="Pookalam_exact.png"):
        
        img = Image.new("RGB", (self.size, self.size), self.BG)
        dr = ImageDraw.Draw(img)

        # 1) Red band (between  rings)
        dr.ellipse(self.circle_bbox(340), fill=self.COL_RED)
        dr.ellipse(self.circle_bbox(320), fill=self.BG)

        # 2) Outer  ring (340–400)
        for poly, col in self.checker_ring(40, 340, 400,
                                           [self.COL_YELLOW, self.COL_ORANGE, self.COL_GREEN]):
            dr.polygon([(int(x), int(y)) for x, y in poly], fill=col, outline=self.OUTLINE)

        # 3) Inner  ring (280–320)
        for poly, col in self.checker_ring(32, 280, 320,
                                           [self.COL_YELLOW, self.COL_ORANGE]):
            dr.polygon([(int(x), int(y)) for x, y in poly], fill=col, outline=self.OUTLINE)

        # 4)  yellow  (circle radius 240)
        dr.ellipse(self.circle_bbox(240), fill=self.COL_YELLOW)

        # 5) Red  around triangles (260–280)
        dr.ellipse(self.circle_bbox(280), fill=self.COL_GREEN)
        dr.ellipse(self.circle_bbox(260), fill=self.COL_YELLOW)

        # 6) Triangle ring
        for poly, col in self.tri_ring():
            dr.polygon([(int(x), int(y)) for x, y in poly], fill=col, outline=self.OUTLINE)

        # 7) Red square at 200
        square_r = 200
        square = [self.polar(square_r, a) for a in (45, 135, 225, 315)]
        dr.polygon([(int(x), int(y)) for x, y in square], fill=self.COL_RED, outline=self.OUTLINE)

        # 8) petals
        for poly in self.petals():
            dr.polygon([(int(x), int(y)) for x, y in poly], fill=self.COL_YELLOW)

        # 9) Central circle + dot
        dr.ellipse(self.circle_bbox(60), fill=self.COL_ORANGE, outline=self.OUTLINE, width=3)
        dr.ellipse(self.circle_bbox(20), fill=self.COL_RED, outline=self.OUTLINE, width=2)

        # 10) Outer border
        dr.ellipse(self.circle_bbox(400), outline=self.OUTLINE, width=3)

        # 11) Outermost triangles
        for poly, col in self.outer_tri_ring():
            dr.polygon([(int(x), int(y)) for x, y in poly], fill=col, outline=self.OUTLINE)

        # Apply grain for a handmade effect
        img = self.add_grain(img, intensity=25)

        # Save to file
        img.save(out_path)
        print(f"✅ Pookalam saved with grain effect → {out_path}")
        return out_path



if __name__ == "__main__":
    PookalamExact(900).draw("C:/Users/mrpri/OneDrive/Desktop/Pookalam_grainy.png")
